In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from retriever import QdrantRetrieverFactory

qs = QdrantRetrieverFactory()

retriever = qs.retriever(collection_name="RAG_Example(RAG_strategies)", fetch_k=3)

In [3]:
retriever.invoke("미드저니")

[Document(metadata={'source': 'docling_outputs\\SPRi AI Brief_8월호_산업동향_F.md', '_id': '99fab946-a7a5-44fc-a827-38cba5e9b3ca', '_collection_name': 'RAG_Example(RAG_strategies)'}, page_content="- n AI 이미지 생성 플랫폼 미드저니(Midjourney)가 2025년 6월 19일 비디오 생성 모델 'V1'을 출시\n- V1은 이미지를 동영상으로 변환하는 모델로, 미드저니 플랫폼에서 제작된 이미지나 외부 이미지를 바탕으로 동영상을 생성하며, '자동' 설정 시에는 모션 프롬프트가 자동으로 생성되고 '수동' 설정을 선택하면 사용자 지시에 따라 사물이나 장면의 움직임을 생성\n- 사용자는 움직임 강도를 피사체와 카메라가 모두 움직이는 '하이 모션(High Motion)'과 카메라는 거의 고정되어 있고 피사체가 천천히 움직이도록 연출 '로우 모션(Low Motion)' 중에서 선택 가능\n- 웹 전용 모델인 V1은 1회 동영상 생성 작업으로 5초 길이의 동영상 4개를 제작하며, 생성된 동영상은 한 번에 4초씩 최대 4회까지 영상 길이를 확장할 수 있도록 허용"),
 Document(metadata={'source': 'docling_outputs\\SPRi AI Brief_8월호_산업동향_F.md', '_id': '3b53a337-de9e-47b0-a2f4-eaedb4519aee', '_collection_name': 'RAG_Example(RAG_strategies)'}, page_content="<!-- image -->\n\n## 기업 ･ 산업\n\n<!-- image -->\n\n## 미드저니, 첫 번째 비디오 생성 AI 모델 'V1' 출시\n\n## KEY Contents\n\n- n 미드저니가 1회 작업으로 5초 길이의 동영상 4개를 제작할 수 있는 비디오 생성 모델 'V1'을 출시하고 여타 비디오 생성 모델보다

In [4]:
from rag import rag

rag_chain = rag(retriever)

In [5]:
rag_chain.invoke({"question": "미드저니"})

"미드저니(Midjourney)는 AI 이미지 생성 플랫폼으로, 2025년 6월 19일 비디오 생성 모델 'V1'을 출시했습니다. V1은 미드저니에서 생성한 이미지나 외부 이미지를 바탕으로 5초 길이의 동영상 4개를 한 번에 제작할 수 있으며, 동영상 길이는 최대 4초씩 4회까지 확장 가능합니다. 사용자는 '하이 모션(High Motion)'과 '로우 모션(Low Motion)' 중 움직임 강도를 선택할 수 있고, 자동 또는 수동 모션 프롬프트 설정이 가능합니다. 미드저니는 2026년에 3D 모델과 실시간 처리 모델을 출시해 통합할 계획이며, V1은 여타 비디오 생성 모델보다 25배 이상 저렴한 가격을 책정했습니다.\n\n**Source**  \n- docling_outputs\\SPRi AI Brief_8월호_산업동향_F.md (p.8)"

# 상태 정의

In [6]:
from typing import TypedDict, Annotated, Sequence, Optional
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages

# create_react_agent에서는 messages를 키로 사용한다.
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    next: Optional[str]  # 다음에 실행할 노드를 지정하는 필드 추가

# create_retriever_tool

retriever를 하나의 도구로 정의

In [7]:
from langchain_core.tools.retriever import create_retriever_tool
from langchain.prompts import PromptTemplate

retriever_tool = create_retriever_tool(retriever=retriever, name="retriever",
                                       description="Search and return information about SPRI AI Brief PDF file. It contains useful information on recent AI trends. The document is published on Dec 2025",
                                       document_prompt=PromptTemplate.from_template(
                                           "<document><content>{page_content}</content><metadata><source>{source}</source><page></page></metadata></document>"
                                            )
                                       )

tools = [retriever_tool]

In [8]:
# query = "에이전틱 RAG의 장점은?"
# result = retriever_tool.invoke(query)
# print(result)

# create_react_agent: Agent생성

create_react_agent는, messages로 대화를 주고 받는게 특징이다.

### retriever_agent 생성

In [9]:
from base import make_system_prompt
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

retriever_agent = create_react_agent(llm, tools=tools,
                                    state_modifier=make_system_prompt(
                                    """You are a specialized research agent focused on retrieving and analyzing information from SPRI AI Brief documents.
                                    
                                    Your primary responsibilities:
                                    1. Search through SPRI AI Brief PDF documents using the retriever tool
                                    2. Find relevant, authoritative information about AI trends, technologies, and industry developments
                                    3. Provide detailed, well-sourced answers based on the retrieved documents
                                    4. Always cite your sources and provide context for the information you share
                                    
                                    You work collaboratively with other agents:
                                    - Researcher agent: Handles web-based research and current information
                                    - Coder agent: Creates visualizations and charts based on your findings
                                    
                                    When retrieving information:
                                    - Use specific, targeted search queries
                                    - Look for the most relevant and recent information
                                    - Provide comprehensive answers with proper context
                                    - Always mention the source of your information
                                    
                                    Remember: You can ONLY use the retriever tool to search SPRI documents. For other types of research, coordinate with the Researcher agent."""), 
                                    checkpointer=memory)



# retriever_node 정의

In [10]:
from langchain_core.messages import AIMessage

def retriever_node(state: AgentState) -> AgentState:
    # invoke 시 분리
    result = retriever_agent.invoke(
        {"messages": state["messages"]}, 
    )

    last_ai = [m for m in result["messages"] if m.type == "ai"][-1]

    return {
        "messages": [AIMessage(content=last_ai.content, name="retriever")],
    }


### Research Agent 생성

TavilySearch 사용

In [11]:
from langchain_tavily import TavilySearch
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState

tavily_tool = TavilySearch(max_results=5)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

research_agent = create_react_agent(llm, tools=[tavily_tool],
                                    state_modifier=make_system_prompt(
                                        "You can only do research. You are working with a chart generator colleague."
                                    ),
                                    checkpointer=MemorySaver()
                                    )
def research_node(state: MessagesState) -> MessagesState:
    result = research_agent.invoke(
        {"messages": state["messages"]},
    )
    last_message = HumanMessage(
        content=result["messages"][-1].content, name="research"
    )
    return {
        "messages": [last_message],
    }

# Chart Generator Agent
PythonREPL 도구를 사용하여 차트를 생성하는 에이전트를 생성합니다. 이 에이전트를 차트를 생성하는 데 사용합니다.

파이썬 repl 도구를 새로 정의합니다.

In [12]:
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

python_repl = PythonREPLTool()

# Python 코드를 실행하는 도구 정의
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        # 주어진 코드를 Python REPL에서 실행하고 결과 반환
        result = python_repl.run(code)
    except BaseException as e:
        return f"Failed to execute code. Error: {repr(e)}"
    # 실행 성공 시 결과와 함께 성공 메시지 반환
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [13]:
code_system_prompt = """
Be sure to use the following font in your code for visualization.

##### 폰트 설정 #####
import platform

# OS 판단
current_os = platform.system()

if current_os == "Windows":
    # Windows 환경 폰트 설정
    font_path = "C:/Windows/Fonts/malgun.ttf"  # 맑은 고딕 폰트 경로
    fontprop = fm.FontProperties(fname=font_path, size=12)
    plt.rc("font", family=fontprop.get_name())
elif current_os == "Darwin":  # macOS
    # Mac 환경 폰트 설정
    plt.rcParams["font.family"] = "AppleGothic"
else:  # Linux 등 기타 OS
    # 기본 한글 폰트 설정 시도
    try:
        plt.rcParams["font.family"] = "NanumGothic"
    except:
        print("한글 폰트를 찾을 수 없습니다. 시스템 기본 폰트를 사용합니다.")

##### 마이너스 폰트 깨짐 방지 #####
plt.rcParams["axes.unicode_minus"] = False  # 마이너스 폰트 깨짐 방지
"""


# Chart Generator Agent 생성
coder_agent = create_react_agent(
    llm,
    [python_repl_tool],
    state_modifier=code_system_prompt,
    checkpointer=MemorySaver()
)

In [14]:
def coder_node(state: MessagesState) -> MessagesState:
    result = coder_agent.invoke(state)

    # 마지막 메시지를 HumanMessage 로 변환
    last_message = HumanMessage(
        content=result["messages"][-1].content, name="coder",
        # config = state["config"]
    )
    
    return {
        # share internal message history of chart agent with other agents
        "messages": [last_message],
        # "config": state["config"]    
        }

# 일반질문

In [15]:
# General Agent 생성 (react_agent로 변경)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent

# General Agent를 react_agent로 생성
general_agent = create_react_agent(
    llm, 
    tools=[],  # 도구 없음 (일반적인 대화만)
    state_modifier=make_system_prompt(
        "You are a helpful general assistant. You can answer general questions, "
        "have conversations, and provide information. You work with other specialized agents: "
        "- Retriever agent: For searching SPRI documents "
        "- Researcher agent: For web research "
        "- Coder agent: For creating charts and visualizations "
        "If you need specialized information, coordinate with the appropriate agent."
    ),
    checkpointer=MemorySaver()
)

def general_node(state: MessagesState) -> MessagesState:
    # react_agent 사용
    result = general_agent.invoke(
        {"messages": state["messages"]},
    )
    
    # 마지막 메시지를 HumanMessage로 변환
    last_message = HumanMessage(
        content=result["messages"][-1].content, name="general",
    )
    
    return {
        "messages": [last_message],
    }


# Agent Supervisor 생성
에이전트를 관리 감독하는 감독자 에이전트를 생성합니다.

In [16]:
from pydantic import BaseModel
from typing import Literal

members = ["Retriever", "Researcher", "Coder", "General LLM"]

options_for_next = ["FINISH"] + members

class RouteResponse(BaseModel):
    next: Literal[*options_for_next]

partial은 olptions와 members를 채우는 값인데, 어떻게 채워야 하는지 고정값이 정해지는것이다.

따라서, `invoke`할때 따로 넣어주지 않아도 되는 값이다.

이중에서 `options_for_next` 중에 하나를 뽑게 되는 prompt, chain이 될 것이다.

In [17]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_openai import ChatOpenAI

# # 시스템 프롬프트 정의: 작업자 간의 대화를 관리하는 감독자 역할
# system_prompt = (
#     "You are a supervisor tasked with managing a conversation between the"
#     " following workers:  {members}. Given the following user request,"
#     " respond with the worker to act next. Each worker will perform a"
#     " task and respond with their results and status."
    
#     " CRITICAL RULE: If a question comes up, you MUST answer it. Don't just send it to FINISH!"
    
#     " IMPORTANT FINISH CONDITIONS:"
#     " - FINISH only when the user's question has been fully answered"
#     " - FINISH only when at least one worker has provided a complete response"
#     " - FINISH only when no additional information or action is needed"
#     " - Do NOT FINISH if the response is incomplete or needs clarification"
#     " - Do NOT FINISH if the user's question is not answered"
#     " WORKER SELECTION RULES:"
#     " - You must select at least one worker before considering FINISH"
#     " - Do not select FINISH immediately - always try a worker first"
#     " - Do not select the same worker more than 2 times in a row"
#     " - If a worker's response is incomplete, select another appropriate worker"
# )

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         MessagesPlaceholder(variable_name="messages"),
#         (
#             "system",
#             "Given the conversation above, who should act next?"
#             "Or should we FINISH? Select one of: {options}"
            
#             " CRITICAL: You MUST answer the user's question before selecting FINISH!"
            
#             " DECISION CRITERIA:"
#             " - If the user's question is fully answered with complete information, select FINISH"
#             " - If the response is incomplete or needs more information, select an appropriate worker"
#             " - If a worker has provided a complete response, consider FINISH"
#             " - If a worker's response is insufficient, select another worker"
#             " - NEVER select FINISH if the user's question is not answered"
            
#             " IMPORTANT RULES:"
#             " - Do not select the same {members} more than 2 times in a row"
#             " - You must select at least one {members} before considering FINISH"
#             " - Only select FINISH when the task is truly complete"
#         )
#     ]
# ).partial(options=str(options_for_next), members=", ".join(members))

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# 시스템 프롬프트 정의: 작업자 간의 대화를 관리하는 감독자 역할
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
    " If a worker has already provided a response, consider if FINISH is appropriate."
    " task and respond with their results and status."
    " You must select at least one worker before considering FINISH."
    " Do not select FINISH immediately - always try a worker first."
    " Do not select the same worker({members}) more than once times in a row."
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            "Or should we FINISH? Select one of: {options}"
            " IMPORTANT: Do not select the same worker({members}) more than once times in a row."
            " IMPORTANT: You must select at least one worker({members}) before considering FINISH."
            " If a worker has already provided a response, consider FINISH."
        )
    ]
).partial(options=str(options_for_next), members=", ".join(members))

In [34]:
def supervisor_agent(state):
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
    supervisor_chain = prompt | llm.with_structured_output(RouteResponse)
    return supervisor_chain.invoke(state)

# 그래프 생성

In [35]:
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver

# 그래프 생성
workflow = StateGraph(AgentState)

# 그래프에 노드 추가
workflow.add_node("Retriever", retriever_node)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", coder_node)
workflow.add_node("General LLM", general_node)

workflow.add_node("Supervisor", supervisor_agent)


# 멤버 노드 > Supervisor 노드로 엣지 추가
for member in members:
    workflow.add_edge(member, "Supervisor")

# 조건부 엣지 추가 (
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END


def get_next(state):
    return state["next"]


# Supervisor 노드에서 조건부 엣지 추가
workflow.add_conditional_edges("Supervisor", get_next, conditional_map)

# 시작점
workflow.add_edge(START, "Supervisor")

# 그래프 컴파일
graph = workflow.compile(checkpointer=MemorySaver())

In [38]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph, stream_graph

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": 123})

# 질문 입력
inputs = {
    "messages": [
        HumanMessage(
            content="안녕?"
        ),
        
    ],"config": config
}

# 그래프 실행
stream_graph(graph, inputs, config)


🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"next":"General LLM"}
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
안녕하세요! 다시 만나서 반가워요. 어떻게 도와드릴까요?
🔄 Node: General LLM 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
안녕하세요! 다시 만나서 반가워요. 어떻게 도와드릴까요?
🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"next":"FINISH"}

In [23]:
# from langchain_core.runnables import RunnableConfig
# from langchain_teddynote.messages import random_uuid, invoke_graph

# # config 설정(재귀 최대 횟수, thread_id)
# config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# # 질문 입력
# inputs = {
#     "messages": [
#         HumanMessage(
#             content="2010년 ~ 2024년까지의 대한민국의 1인당 GDP 추이를 그래프로 시각화 해주세요."
#         ),
        
#     ],"config": config
# }

# # 그래프 실행
# invoke_graph(graph, inputs, config)

In [24]:
# 수정된 코드 테스트
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": 123})

# 질문 입력
inputs = {
    "messages": [
        HumanMessage(
            content="미드저니 신버전은? SPRI에서 찾아줘"
        )
    ],
}

# 그래프 실행
invoke_graph(graph, inputs, config)



🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
next:
Retriever

🔄 Node: agent in [Retriever] 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================
Tool Calls:
  retriever (call_RTupIlWikN0XlkoU4YHSZRD8)
 Call ID: call_RTupIlWikN0XlkoU4YHSZRD8
  Args:
    query: Midjourney new version

🔄 Node: tools in [Retriever] 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================= Tool Message =================================
Name: retriever

<document><content>| 12월 | 3~5일   | (소프트웨이브 2025)10회대한민국소프트웨어대전                                     | 서울, 강남                | www.k-softwave.com                                       |
| 12월 | 10~11일 | AI Summit New York                                                                | 미국, 뉴욕                | newyork.theaisummit.com                                  |</content><metadata><source>docling_outputs\SPRi AI Brief_8월

In [25]:
# 수정된 코드 테스트
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": 123})

# 질문 입력
inputs = {
    "messages": [
        HumanMessage(
            content="이전 나의 질문은?"
        )
    ],
}

# 그래프 실행
invoke_graph(graph, inputs, config)



🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
next:
General LLM

🔄 Node: agent in [General LLM] 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

이전 질문은 "미드저니 신버전은? SPRI에서 찾아줘"였습니다.

🔄 Node: General LLM 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================ Human Message =================================
Name: general

이전 질문은 "미드저니 신버전은? SPRI에서 찾아줘"였습니다.

🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
next:
FINISH


In [26]:
# 수정된 코드 테스트
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph, stream_graph

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": 123})

# 질문 입력
inputs = {
    "messages": [
        HumanMessage(
            content="미드저니 신버전"
        )
    ],
}

# 그래프 실행
invoke_graph(graph, inputs, config)



🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
next:
FINISH


In [27]:
# 수정된 코드 테스트
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph, stream_graph

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": 123})

# 질문 입력
inputs = {
    "messages": [
        HumanMessage(
            content="안녕??"
        )
    ],
}

# 그래프 실행
stream_graph(graph, inputs, config)



🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"next":"FINISH"}

In [28]:
# 실제 챗봇 환경 시뮬레이션 - 같은 질문 반복 테스트
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

print("=== 실제 챗봇 환경 시뮬레이션 ===")
print("같은 질문이 여러 번 들어와도 매번 답변해야 합니다.")

# 첫 번째 질문
print("\n1️⃣ 첫 번째 질문:")
config1 = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
inputs1 = {
    "messages": [
        HumanMessage(content="안녕?")
    ]
}
result1 = invoke_graph(graph, inputs1, config1)

print("\n" + "="*50)

# 두 번째 질문 (같은 내용)
print("\n2️⃣ 두 번째 질문 (같은 내용):")
config2 = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
inputs2 = {
    "messages": [
        HumanMessage(content="안녕?")
    ]
}
result2 = invoke_graph(graph, inputs2, config2)

print("\n" + "="*50)

# 세 번째 질문 (같은 내용)
print("\n3️⃣ 세 번째 질문 (같은 내용):")
config3 = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
inputs3 = {
    "messages": [
        HumanMessage(content="안녕?")
    ]
}
result3 = invoke_graph(graph, inputs3, config3)


=== 실제 챗봇 환경 시뮬레이션 ===
같은 질문이 여러 번 들어와도 매번 답변해야 합니다.

1️⃣ 첫 번째 질문:

🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
next:
General LLM

🔄 Node: agent in [General LLM] 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

안녕! 어떻게 도와줄까?

🔄 Node: General LLM 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================ Human Message =================================
Name: general

안녕! 어떻게 도와줄까?

🔄 Node: Supervisor 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
next:
General LLM

🔄 Node: agent in [General LLM] 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

안녕! 무엇을 도와줄까? 궁금한 점이나 필요한 게 있으면 말해줘!

🔄 Node: General LLM 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================ Human Message =================================
Name: general

안녕! 무엇을 도와줄까

GraphRecursionError: Recursion limit of 10 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
# 메모리 없는 그래프로 테스트 (각 질문이 완전히 독립적)
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver

# 메모리 없는 그래프 생성
workflow_no_memory = StateGraph(AgentState)

# 그래프에 노드 추가
workflow_no_memory.add_node("Retriever", retriever_node)
workflow_no_memory.add_node("Researcher", research_node)
workflow_no_memory.add_node("Coder", coder_node)
workflow_no_memory.add_node("General LLM", general_node)
workflow_no_memory.add_node("Supervisor", supervisor_agent)

# 멤버 노드 > Supervisor 노드로 엣지 추가
for member in members:
    workflow_no_memory.add_edge(member, "Supervisor")

# 조건부 엣지 추가
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END

def get_next(state):
    return state["next"]

# Supervisor 노드에서 조건부 엣지 추가
workflow_no_memory.add_conditional_edges("Supervisor", get_next, conditional_map)

# 시작점
workflow_no_memory.add_edge(START, "Supervisor")

# 메모리 없는 그래프 컴파일 (수정된 general_node 반영)
graph_no_memory = workflow_no_memory.compile()  # checkpointer 없음

print("=== 메모리 없는 그래프 재생성 완료 ===")
print("이제 각 질문이 완전히 독립적으로 처리됩니다.")
print("general_node도 react_agent를 사용하여 무한루프를 방지합니다.")


In [ ]:
# 메모리 없는 그래프로 같은 질문 반복 테스트
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

print("=== 메모리 없는 그래프로 같은 질문 반복 테스트 ===")

# 첫 번째 질문
print("\n1️⃣ 첫 번째 질문:")
inputs1 = {
    "messages": [
        HumanMessage(content="안녕?")
    ]
}
result1 = invoke_graph(graph_no_memory, inputs1)

print("\n" + "="*50)

# 두 번째 질문 (같은 내용)
print("\n2️⃣ 두 번째 질문 (같은 내용):")
inputs2 = {
    "messages": [
        HumanMessage(content="안녕?")
    ]
}
result2 = invoke_graph(graph_no_memory, inputs2)

print("\n" + "="*50)

# 세 번째 질문 (같은 내용)
print("\n3️⃣ 세 번째 질문 (같은 내용):")
inputs3 = {
    "messages": [
        HumanMessage(content="안녕?")
    ]
}
result3 = invoke_graph(graph_no_memory, inputs3)


In [ ]:
# 수정된 general_node 테스트 (무한루프 방지)
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

print("=== 수정된 general_node 테스트 ===")
print("이제 general_node도 react_agent를 사용하여 무한루프를 방지합니다.")

# 새로운 thread_id로 config 설정
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = {
    "messages": [
        HumanMessage(
            content="안녕?"
        )
    ]
}

# 수정된 그래프로 실행
result = invoke_graph(graph, inputs, config)
